In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages.ai import AIMessage

In [ ]:
import operator
from utility import generate_tools, extract_relevant_info
from typing import TypedDict, Annotated, Sequence, Optional, Dict, List
from langchain.tools.render import render_text_description

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
_, model_tools = generate_tools("resume.json")

In [ ]:
ollama_llm = ChatOllama(model="llama3.1:latest")

In [ ]:
system_prompt = """
You are a Job Application Agent tasked to apply for jobs online with execution rules.

Instructions:
1. Perform one action ( tool Call | Observation | Analysis ) per step.
2. Follow Call (tool) - Observe (Tool output) - Analyse (Use Observation for next action selection) cycle.
3. Fill state values like elements_summary | elementsOfInterest | actions_planned to keep track of progress.
3. Only interact with essential elements.
4. Element priority is as follows:
   a) Required form fields
   b) Continue/Next buttons
   c) Submit buttons
5. For login pages:
   - First fill username
   - Then fill password
   - Finally click sign-in
6. Read Tool description to understand how to interact with tool.

Tools Available:
{tools}

Tool Specific Instruction:
1. Resume Query Tool: To retrieve login credentials for login page, use the keyword `login`.

General Process step-by-step:
1. Navigate to the job link.
2. Retrieve Page title and elements summary.
3. Analyze the elements summary along with page title to determine the elements of interest.
4. For Identified elements of interest containing action 'fill', Retrieve text with resume query tool.
5. Use the appropriate tool to fill or click the elements of interest along with the relevant inputs.
6. Repeat until the application is complete or no further progress is possible.

Special Instructions:
1. Do not call multiple tools in a single step.
2. Do not invent information—always use `query_resume` with the correct keyword to retrieve required data.
3. If you encounter an error or unexpected result in a tool call, reason the error before proceeding.
"""

In [ ]:
class AgentState(TypedDict):
    elements_summary: Optional[Dict]
    elementsOfInterest: Optional[Dict]
    actions_planned: Optional[Dict]
    messages: Annotated[Sequence[HumanMessage | AIMessage], operator.add]
    remaining_steps: int


In [ ]:
job_agent = create_react_agent(
    model=ollama_llm,
    tools=model_tools,
    prompt=system_prompt.format(tools=render_text_description(model_tools)),
    state_schema=AgentState
)

In [ ]:
job_link = "https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4037021039&discover=recommended&discoveryOrigin=JOBS_HOME_JYMBII"

In [ ]:
initial_state = dict(
    elements_summary={},
    elementsOfInterest={},
    actions_planned={},
    messages=[HumanMessage(content=f"For the job_link: {job_link} \nUse the provided tools appropriately to complete job application.")],
    remaining_steps=10
)

In [ ]:
async for chunk in job_agent.astream(
    initial_state,
    stream_mode="updates"
):
    print(await extract_relevant_info(chunk))

In [ ]:
# [{'type': 'AIMessage', 'content': "To answer your question, I will follow these steps:\n\n1. Navigate to the job link.\n2. Retrieve Page title and elements summary.\n3. Analyze the elements summary along with page title to determine the elements of interest.\n4. For Identified elements of interest containing action 'fill', Retrieve text with resume query tool.\n5. Use the appropriate tool to fill or click the elements of interest along with the relevant inputs.\n\nHere is a JSON that best answers your prompt:\n\n```json", 'tool_calls': [{'tool_name': 'navigate_browser', 'input': {'url': 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4037021039&discover=recommended&discoveryOrigin=JOBS_HOME_JYMBII'}}, {'tool_name': 'get_all_elements', 'input': {}}]}]
# [{'type': 'ToolMessage', 'content': {'Page Title': '', 'Page Elements Summary': []}, 'tool_name': 'get_all_elements'}]
# [{'type': 'ToolMessage', 'content': 'Navigating successfull.', 'tool_name': 'navigate_browser'}]
# [{'type': 'AIMessage', 'content': "We can't get the page elements summary since it's empty. Let's navigate to the page and try again.\n\n\n", 'tool_calls': [{'tool_name': 'get_all_elements', 'input': {}}]}]
# [{'type': 'ToolMessage', 'content': {'Page Title': 'LinkedIn Login, Sign in | LinkedIn', 'Page Elements Summary': [{'selector': 'button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary', 'attributes': {'id': None, 'aria-label': None, 'type': None, 'maxlength': None}, 'possible_actions': ['click']}, {'selector': 'button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary', 'attributes': {'id': None, 'aria-label': None, 'type': None, 'maxlength': None}, 'possible_actions': ['click']}, {'selector': 'button.sign-in-with-apple-button', 'attributes': {'id': None, 'aria-label': 'Sign in with Apple', 'type': 'button', 'maxlength': None}, 'possible_actions': ['click']}, {'selector': '#username', 'attributes': {'id': 'username', 'aria-label': 'Email or phone', 'type': 'email', 'maxlength': None}, 'possible_actions': ['fill']}, {'selector': '#password', 'attributes': {'id': 'password', 'aria-label': 'Password', 'type': 'password', 'maxlength': None}, 'possible_actions': ['fill']}, {'selector': 'button.btn__primary--large.from__button--floating', 'attributes': {'id': None, 'aria-label': 'Sign in', 'type': 'submit', 'maxlength': None}, 'possible_actions': ['click']}, {'selector': '#btn-resend-otp', 'attributes': {'id': 'btn-resend-otp', 'aria-label': 'Resend email', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': '#otp-cancel-button', 'attributes': {'id': 'otp-cancel-button', 'aria-label': 'Back', 'type': None, 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__button', 'attributes': {'id': None, 'aria-label': None, 'type': None, 'maxlength': None}, 'possible_actions': ['click']}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'العربية (Arabic) 1 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'বাংলা (Bangla) 2 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Čeština (Czech) 3 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Dansk (Danish) 4 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Deutsch (German) 5 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Ελληνικά (Greek) 6 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link.language-selector__link--selected', 'attributes': {'id': None, 'aria-label': 'English (English) 7 of 36 selected', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Español (Spanish) 8 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'فارسی (Persian) 9 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Suomi (Finnish) 10 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Français (French) 11 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'हिंदी (Hindi) 12 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Magyar (Hungarian) 13 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Bahasa Indonesia (Indonesian) 14 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Italiano (Italian) 15 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'עברית (Hebrew) 16 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '日本語 (Japanese) 17 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '한국어 (Korean) 18 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'मराठी (Marathi) 19 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Bahasa Malaysia (Malay) 20 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Nederlands (Dutch) 21 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Norsk (Norwegian) 22 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'ਪੰਜਾਬੀ (Punjabi) 23 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Polski (Polish) 24 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Português (Portuguese) 25 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Română (Romanian) 26 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Русский (Russian) 27 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Svenska (Swedish) 28 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'తెలుగు (Telugu) 29 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'ภาษาไทย (Thai) 30 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Tagalog (Tagalog) 31 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Türkçe (Turkish) 32 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Українська (Ukrainian) 33 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Tiếng Việt (Vietnamese) 34 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '简体中文 (Chinese (Simplified)) 35 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '正體中文 (Chinese (Traditional)) 36 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}]}, 'tool_name': 'get_all_elements'}]
# [{'type': 'AIMessage', 'content': 'Elements of interest: \n1. username input field\n2. password input field\n3. login button\n\nAction Plan:\n1. Fill username input field\n2. Fill password input field\n3. Click on the login button\n\nTool call to fill username input field:\n\n', 'tool_calls': [{'tool_name': 'fill_element', 'input': {'selector': '#username', 'text': 'linkedin email'}}]}]
# [{'type': 'ToolMessage', 'content': 'Filled element #username with text linkedin email', 'tool_name': 'fill_element'}]
# [{'type': 'AIMessage', 'content': '', 'tool_calls': [{'tool_name': 'fill_element', 'input': {'selector': '#password', 'text': 'linkedin password'}}]}]
# [{'type': 'ToolMessage', 'content': 'Filled element #password with text linkedin password', 'tool_name': 'fill_element'}]
# [{'type': 'AIMessage', 'content': '', 'tool_calls': [{'tool_name': 'click_element', 'input': {'selector': 'button.btn__primary--large.from__button--floating'}}]}]
# [{'type': 'ToolMessage', 'content': "Clicked element 'button.btn__primary--large.from__button--floating'", 'tool_name': 'click_element'}]
# [{'type': 'AIMessage', 'content': 'elements_summary = [{"selector": "button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary", "attributes": {"id": null, "aria-label": null, "type": null, "maxlength": null}, "possible_actions": ["click"]}, {"selector": "button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary", "attributes": {"id": null, "aria-label": null, "type": null, "maxlength": null}, "possible_actions": ["click"]}, {"selector": "button.sign-in-with-apple-button", "attributes": {"id": null, "aria-label": "Sign in with Apple", "type": "button", "maxlength": null}, "possible_actions": ["click"]}, {"selector": "#username", "attributes": {"id": "username", "aria-label": "Email or phone", "type": "email", "maxlength": null}, "possible_actions": []}, {"selector": "#password", "attributes": {"id": "password", "aria-label": "Password", "type": "password", "maxlength": null}, "possible_actions": []}, {"selector": "button.btn__primary--large.from__button--floating", "attributes": {"id": null, "aria-label": "Sign in", "type": "submit", "maxlength": null}, "possible_actions": ["click"]}, {"selector": "#btn-resend-otp", "attributes": {"id": "btn-resend-otp", "aria-label": "Resend email", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "#otp-cancel-button", "attributes": {"id": "otp-cancel-button", "aria-label": "Back", "type": null, "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__button", "attributes": {"id": null, "aria-label": null, "type": null, "maxlength": null}, "possible_actions": ["click"]}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "العربية (Arabic) 1 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "বাংলা (Bangla) 2 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Čeština (Czech) 3 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Dansk (Danish) 4 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Deutsch (German) 5 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Ελληνικά (Greek) 6 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link.language-selector__link--selected", "attributes": {"id": null, "aria-label": "English (English) 7 of 36 selected", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Español (Spanish) 8 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "मराठी (Marathi) 19 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Bahasa Malaysia (Malay) 20 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Nederlands (Dutch) 21 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Norsk (Norwegian) 22 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "ਪੰਜਾਬੀ (Punjabi) 23 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Polski (Polish) 24 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Português (Portuguese) 25 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Română (Romanian) 26 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Русский (Russian) 27 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Svenska (Swedish) 28 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "తెలుగు (Telugu) 29 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "ภาษาไทย (Thai) 30 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Tagalog (Tagalog) 31 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Türkçe (Turkish) 32 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Українська (Ukrainian) 33 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Tiếng Việt (Vietnamese) 34 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "简体中文 (Chinese (Simplified)) 35 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "正體中文 (Chinese (Traditional)) 36 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}]\n\nWe are now on the LinkedIn login page.', 'tool_calls': []}]

In [ ]:
# How to use request library to chat with locally served ollama model.
# https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion
# https://github.com/ollama/ollama
# import requests

# url = "http://localhost:11434/api/generate"
# data = {
#     "model": "gemma3:12b-it-qat",
#     "prompt": "Why is the sky blue?",
#     "stream": False
# }

# response = requests.post(url, json=data)
# print(response.text)
